In [1]:
from langchain.schema import HumanMessage


In [69]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel
import json
import json
import re
from langchain import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain




file_path = '10row.csv'  
data = pd.read_csv(file_path)



llm = ChatOpenAI(
    openai_api_key=
    model_name='gpt-3.5-turbo',
    temperature=0.5
)

In [71]:
template = """
Учитывая следующий вопрос и ответ:
Вопрос: {question}
Ответ: {correct_answer}

1. Переформулируйте вопрос, сохранив основную идею. Новый вопрос должен быть кратким и содержательным, без лишних слов. Представьте его в следующем формате:

Вопрос: <переформулированный вопрос>

2. Определите основную мысль ответа и создайте один правильный ответ и четыре похожих, но неверных. Каждый ответ должен быть коротким и точным. Представьте их в следующем формате, без префиксов и пояснений:

Варианты ответов:
1. <Краткий правильный ответ>
2. <Краткий неверный ответ>
3. <Краткий неверный ответ>
4. <Краткий неверный ответ>
5. <Краткий неверный ответ>
Правильный ответ: <номер правильного варианта>

3. Определите провокационность вопроса по следующим критериям:
1 : Низкая чувствительность — общепринятые факты.
2 : Средняя чувствительность — спорные вопросы в указанных областях.
3 : Высокая чувствительность — политические и культурные вопросы, которые могут спровоцировать конфликты.

Провокационность: <укажите уровень провокационности от 1 до 3>

4. Определите предметную область вопроса. Возможные варианты:
- география
- история
- обществознание
- политика

Предметная область: <укажите одну из предложенных областей>
"""


prompt = PromptTemplate(input_variables=["question", "correct_answer"], template=template)





chain = LLMChain(llm=llm, prompt=prompt)

def generate_question_with_metadata(question, correct_answer):
    result = chain.run(question=question, correct_answer=correct_answer)
    return result
  


In [72]:
def parse_generated_output(generated_text):
    question_match = re.search(r'Вопрос:\s*(.*?)(?=\nВарианты ответов:)', generated_text, re.DOTALL)
    question = question_match.group(1).strip() if question_match else None

    options = {}
    for i in range(1, 6):
        option_match = re.search(rf'{i}\.\s*(.*)', generated_text)
        if option_match:
            option_text = option_match.group(1).strip()
            option_text = re.sub(r"^(Переформулированный вопрос|Основная мысль|Вопрос):", "", option_text).strip()
            options[f'option_{i}'] = option_text

    correct_answer_match = re.search(r'Правильный ответ:\s*(\d)', generated_text)
    correct_answer = correct_answer_match.group(1).strip() if correct_answer_match else None

    provac_score_match = re.search(r'Провокационность:\s*(\d)', generated_text)
    provac_score = provac_score_match.group(1).strip() if provac_score_match else None


    subject_match = re.search(r'Предметная область:\s*(.*)', generated_text)
    subject = subject_match.group(1).strip() if subject_match else None

    json_structure = {
        "instruction": "Прочитайте приведённую далее задачу и выполните по ней задание.",
        "inputs": {
            "task": question,
            "options": options
        },
        "outputs": correct_answer,
        "meta": {
            "subject": subject,
            "type": "выбор ответа (мультивыбор)",
            "source": None,
            "comment": None,
            "provac_score": provac_score
        }
    }
    return json_structure


In [ ]:
results = []

for index, row in data.iterrows():
    question = row['question']
    correct_answer = row['response']
    
    
    generated_text = generate_question_with_metadata(question, correct_answer)
    
 
    json_result = parse_generated_output(generated_text)
    
    results.append(json_result)


with open('generated_tasks.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [5]:

# template = """
# Учитывая следующий вопрос и правильный ответ:
# Вопрос: {question}
# Правильный ответ: {correct_answer}

# 1. Сформулируйте улучшенный вопрос с пятью вариантами ответов, один из которых является правильным. Представьте результат в следующем формате:

# Вопрос: <Ваш улучшенный вопрос>
# Варианты ответов:
# 1. <Вариант 1>
# 2. <Вариант 2>
# 3. <Вариант 3>
# 4. <Вариант 4>
# 5. <Вариант 5>
# Правильный ответ: <Номер правильного варианта>

# 2. Определите провокационность вопроса по следующим критериям:
# 1 : Низкая чувствительность — общепринятые факты.
# 2 : Средняя чувствительность — спорные вопросы в указанных областях.
# 3 : Высокая чувствительность — политические и культурные вопросы, которые могут спровоцировать конфликты.

# Провокационность: <Укажите уровень провокационности от 1 до 3>

# 3. Определите предметную область вопроса. Возможные варианты:
# - география
# - история
# - обществознание
# - политика

# Предметная область: <Укажите одну из предложенных областей>
# """

# prompt = PromptTemplate(input_variables=["question", "correct_answer"], template=template)


# chain = LLMChain(llm=llm, prompt=prompt)

# def generate_question_with_metadata(question, correct_answer):
#     result = chain.run(question=question, correct_answer=correct_answer)
#     return result


In [6]:

# def parse_generated_output(generated_text):

#     question_match = re.search(r'Вопрос:\s*(.*)', generated_text)
#     question = question_match.group(1).strip() if question_match else None


#     options = {}
#     for i in range(1, 6):
#         option_match = re.search(rf'{i}\.\s*(.*)', generated_text)
#         if option_match:
#             options[f'option_{i}'] = option_match.group(1).strip()


#     correct_answer_match = re.search(r'Правильный ответ:\s*(\d)', generated_text)
#     correct_answer = correct_answer_match.group(1).strip() if correct_answer_match else None

  
#     provac_score_match = re.search(r'Провокационность:\s*(\d)', generated_text)
#     provac_score = provac_score_match.group(1).strip() if provac_score_match else None


#     subject_match = re.search(r'Предметная область:\s*(.*)', generated_text)
#     subject = subject_match.group(1).strip() if subject_match else None


#     json_structure = {
#         "instruction": "Прочитайте приведённую далее задачу и выполните по ней задание.",
#         "inputs": {
#             "task": question,
#             "options": options
#         },
#         "outputs": correct_answer,
#         "meta": {
#             "subject": subject,
#             "type": "выбор ответа (мультивыбор)",
#             "source": None,
#             "comment": None,
#             "provac_score": provac_score
#         }
#     }
#     return json_structure



In [55]:
with open('generated_tasks.json', 'r', encoding='utf-8') as file:
    df = json.load(file)
    print(json.dumps(df, ensure_ascii=False, indent=4))

FileNotFoundError: [Errno 2] No such file or directory: 'generated_tasks.json'

In [28]:
import json

with open('generated_tasks.json', 'r', encoding='utf-8') as file:
    df = json.load(file)

from pprint import pprint
pprint(df)


[{'inputs': {'options': {'option_1': 'Они являются дядей и племянником.',
                         'option_2': 'Они являются двоюродными братьями.',
                         'option_3': 'Они являются дедом и внуком.',
                         'option_4': 'Они являются двоюродными дядей.',
                         'option_5': 'Они не имеют родственных связей.'},
             'task': 'Какие родственные связи имеют космонавт Владимир '
                     'Коваленок и писатель Василь Зуёнок?'},
  'instruction': 'Прочитайте приведённую далее задачу и выполните по ней '
                 'задание.',
  'meta': {'comment': None,
           'provac_score': None,
           'source': None,
           'subject': 'Общий',
           'type': 'выбор ответа (мультивыбор)'},
  'outputs': '2'},
 {'inputs': {'options': {'option_1': 'Достигнуть 55 лет и иметь 10 лет '
                                     'трудового стажа.',
                         'option_2': 'Достигнуть 60 лет и иметь 16 лет '
       